In [1]:
import sys
import os
import json
import pandas as pd


# Change dir to root dir
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
os.chdir(parent_dir)
print("Current Working Directory:", os.getcwd())

from src.data import generate_df, convert_to_gdf, generate_gdf, preprocess_df
from src.plot import plot_data
from src.model import light_gbm, predict_light_gbm_model
from src.data import generate_df, generate_gdf, preprocess_df, preprocess_gdf, filter_nam_outside_vri, get_nam_outside_vri_nearest_station
from src.scripts.generateNamCSV import generate_nam_csv
from src.scripts.generateElevationCSV import generate_elevation_csv
from src.analysis import custom_groupby, find_outliers_iqr


Current Working Directory: /home/khidayat/private/grade/SDGE_NAM_Analysis


In [2]:
with open('config/data_params.json') as fh:
        data_params = json.load(fh)    

raw_data_path = [os.path.join('./data/raw', file_path) for file_path in data_params["raw_data"]]
modified_data_path = [os.path.join('./data/modified', file_path) for file_path in data_params["modified_data"]]
output_model_path = [os.path.join('./data/modified', file_path) for file_path in data_params["model_prediction"]]

In [3]:
# Reading raw data
gis_weather_station, src_vri_snapshot, nam, windspeed_snapshot = generate_df(raw_data_path) 
gis_weather_station, windspeed_snapshot = preprocess_df(gis_weather_station, windspeed_snapshot)

# Reading filtered data with elevation from API
gis_weather_station_with_elevation, nam_with_elevation = generate_df(modified_data_path)
gis_weather_station_with_elevation_gpd, src_vri_snapshot_gpd, nam_with_elevation_gpd = generate_gdf(
    gis_weather_station_with_elevation, src_vri_snapshot, nam_with_elevation)

# Readiing data from LightGBM model
nam_within_vri_prediction, nam_outside_vri_prediction = generate_df(output_model_path)

Weather Station CRS:    EPSG:4326
VRI Polygon CRS:        EPSG:4326
NAM CRS:                EPSG:4326


/home/khidayat/private/grade/SDGE_NAM_Analysis/src/data.py:9: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  return [pd.read_csv(file) for file in input_list]


In [4]:
nam_within_vri_prediction.head()

,objectid,weatherstationcode,weatherstationname,scadartuid,structureid_left,nwszone,district_left,thomasbrospagegrid,constructionstatus,creationuser,...,shape_srid_right,snapshot_date,nam_geometry,station_geometry,polygon_geometry,nam_distance_from_station_km,month,day_of_year,abs_wind_speed_error,abs_wind_speed_error_pred
0,1,CBD,Carlsbad,5158.0,P124785,Coastal-243,6.0,1126-G1,A,seu_gis_elec,...,4326,2024-03-20,POINT (-117.328186 33.142284),POINT (-117.3271713424031 33.13734577077235),"MULTIPOLYGON Z (((-117.328519 33.134906 0, -11...",0.555806,12,345,3.285754,5.296421
1,1,CBD,Carlsbad,5158.0,P124785,Coastal-243,6.0,1126-G1,A,seu_gis_elec,...,4326,2024-03-20,POINT (-117.31195 33.142193),POINT (-117.3271713424031 33.13734577077235),"MULTIPOLYGON Z (((-117.328519 33.134906 0, -11...",1.518577,12,345,3.148903,4.653384
2,1,CBD,Carlsbad,5158.0,P124785,Coastal-243,6.0,1126-G1,A,seu_gis_elec,...,4326,2024-03-20,POINT (-117.328094 33.15589),POINT (-117.3271713424031 33.13734577077235),"MULTIPOLYGON Z (((-117.328519 33.134906 0, -11...",2.058487,12,345,3.598310,5.359173
3,1,CBD,Carlsbad,5158.0,P124785,Coastal-243,6.0,1126-G1,A,seu_gis_elec,...,4326,2024-03-20,POINT (-117.31183 33.155807),POINT (-117.3271713424031 33.13734577077235),"MULTIPOLYGON Z (((-117.328519 33.134906 0, -11...",2.498172,12,345,3.535970,4.937996
4,1,CBD,Carlsbad,5158.0,P124785,Coastal-243,6.0,1126-G1,A,seu_gis_elec,...,4326,2024-03-20,POINT (-117.31174 33.169407),POINT (-117.3271713424031 33.13734577077235),"MULTIPOLYGON Z (((-117.328519 33.134906 0, -11...",3.836187,12,345,3.631954,5.742182


In [5]:
nam_within_vri_prediction.columns

Index(['objectid', 'weatherstationcode', 'weatherstationname', 'scadartuid',
       'structureid_left', 'nwszone', 'district_left', 'thomasbrospagegrid',
       'constructionstatus', 'creationuser', 'datecreated', 'datemodified',
       'lastuser', 'structureguid', 'symbolrotation', 'latitude_left',
       'longitude_left', 'elevation', 'twinguid', 'hftd_left', 'zone1idc_left',
       'hftdidc_left', 'gdb_geomattr_data', 'globalid_left', 'shape_left',
       'shape_srid_left', 'snapshot_date_x', 'geometry', 'station_elevation_m',
       'station_date', 'station_wind_speed', 'snapshot_date_y',
       'latitude_right', 'longitude_right', 'nam_date', 'nam_wind_speed',
       'nam_elevation_m', 'name', 'tessellate', 'extrude', 'visibility',
       'globalid_right', 'anemometer', 'anemometercode', 'circuit',
       'district_right', 'secdevice', 'structureid_right', 'tlid',
       'gust_99pct', 'gust_95pct', 'gust_max', 'vri_risk', 'loaddate',
       'county', 'hftd_right', 'hftdidc_right',

In [6]:
# # Select only numeric columns
# numeric_columns = nam_within_vri_prediction.select_dtypes(include=['number']).columns

# # Group by polygon and compute the mean only for numeric columns
# nam_grouped_within_vri_prediction = nam_within_vri_prediction.groupby('polygon_geometry')[numeric_columns].mean()


# nam_grouped_within_vri_prediction = nam_within_vri_prediction.groupby('polygon_geometry').mean()

In [7]:
nam_within_vri_prediction_gpd = convert_to_gdf(nam_within_vri_prediction)
nam_within_vri_prediction_gpd

,objectid,weatherstationcode,weatherstationname,scadartuid,structureid_left,nwszone,district_left,thomasbrospagegrid,constructionstatus,creationuser,...,shape_srid_right,snapshot_date,nam_geometry,station_geometry,polygon_geometry,nam_distance_from_station_km,month,day_of_year,abs_wind_speed_error,abs_wind_speed_error_pred
0,1,CBD,Carlsbad,5158.0,P124785,Coastal-243,6.0,1126-G1,A,seu_gis_elec,...,4326,2024-03-20,POINT (-117.328186 33.142284),POINT (-117.3271713424031 33.13734577077235),"MULTIPOLYGON Z (((-117.328519 33.134906 0, -11...",0.555806,12,345,3.285754,5.296421
1,1,CBD,Carlsbad,5158.0,P124785,Coastal-243,6.0,1126-G1,A,seu_gis_elec,...,4326,2024-03-20,POINT (-117.31195 33.142193),POINT (-117.3271713424031 33.13734577077235),"MULTIPOLYGON Z (((-117.328519 33.134906 0, -11...",1.518577,12,345,3.148903,4.653384
2,1,CBD,Carlsbad,5158.0,P124785,Coastal-243,6.0,1126-G1,A,seu_gis_elec,...,4326,2024-03-20,POINT (-117.328094 33.15589),POINT (-117.3271713424031 33.13734577077235),"MULTIPOLYGON Z (((-117.328519 33.134906 0, -11...",2.058487,12,345,3.598310,5.359173
3,1,CBD,Carlsbad,5158.0,P124785,Coastal-243,6.0,1126-G1,A,seu_gis_elec,...,4326,2024-03-20,POINT (-117.31183 33.155807),POINT (-117.3271713424031 33.13734577077235),"MULTIPOLYGON Z (((-117.328519 33.134906 0, -11...",2.498172,12,345,3.535970,4.937996
4,1,CBD,Carlsbad,5158.0,P124785,Coastal-243,6.0,1126-G1,A,seu_gis_elec,...,4326,2024-03-20,POINT (-117.31174 33.169407),POINT (-117.3271713424031 33.13734577077235),"MULTIPOLYGON Z (((-117.328519 33.134906 0, -11...",3.836187,12,345,3.631954,5.742182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102827,159,CLM,Creelman,1996.0,Z104188,Inland Valley-250,3.0,1172-G2,A,seu_gis_elec,...,4326,2024-03-20,POINT (-116.8421 33.02989),POINT (-116.8709969702836 33.01564947775751),"MULTIPOLYGON Z (((-116.908853 33.012693 0, -11...",3.127836,12,359,5.978363,11.228269
102828,159,CLM,Creelman,1996.0,Z104188,Inland Valley-250,3.0,1172-G2,A,seu_gis_elec,...,4326,2024-03-20,POINT (-116.858154 33.043648),POINT (-116.8709969702836 33.01564947775751),"MULTIPOLYGON Z (((-116.908853 33.012693 0, -11...",3.328915,12,359,5.092426,7.685208
102829,159,CLM,Creelman,1996.0,Z104188,Inland Valley-250,3.0,1172-G2,A,seu_gis_elec,...,4326,2024-03-20,POINT (-116.841934 33.0435),POINT (-116.8709969702836 33.01564947775751),"MULTIPOLYGON Z (((-116.908853 33.012693 0, -11...",4.112461,12,359,5.454389,7.881753
102830,159,CLM,Creelman,1996.0,Z104188,Inland Valley-250,3.0,1172-G2,A,seu_gis_elec,...,4326,2024-03-20,POINT (-116.98926 32.935913),POINT (-116.8709969702836 33.01564947775751),"MULTIPOLYGON Z (((-116.871922 33.01855 0, -116...",14.156803,12,359,14.118581,14.601793


In [8]:
# Wind speed error difference to evaluate the LightGBM model
nam_within_vri_prediction_gpd['wind_speed_error_diff'] = (nam_within_vri_prediction_gpd['abs_wind_speed_error'] - 
                                                     nam_within_vri_prediction_gpd['abs_wind_speed_error_pred']).abs()

agg_dict_within = {
    'abs_wind_speed_error': 'mean',
    'wind_speed_error_diff': 'mean',
    'nam_distance_from_station_km': 'mean'
}

# Group the data based on each NAM points
nam_mae_within = custom_groupby(nam_within_vri_prediction_gpd, ['geometry'], agg_dict_within)
nam_mae_within['distance_weight_error'] = nam_mae_within['abs_wind_speed_error'] * nam_mae_within['nam_distance_from_station_km']
nam_mae_within.head()

,geometry,abs_wind_speed_error,wind_speed_error_diff,nam_distance_from_station_km,distance_weight_error
0,POINT (-117.05798 32.6016),6.565758,1.434867,3.088612,20.279076
1,POINT (-117.09667 32.54195),8.642320,2.520737,1.200859,10.378207
2,POINT (-116.90043 32.55813),13.760803,2.887206,5.229643,71.964082
3,POINT (-116.98088 32.68479),8.355274,2.915239,2.317221,19.361013
4,POINT (-116.8699 32.7139),6.892383,2.128632,3.010902,20.752289


In [9]:
nam_points_to_update = nam_mae_within.sort_values(
    by='abs_wind_speed_error', ascending=False
).head(20)['geometry'].tolist()
nam_points_to_update

[<POINT (-116.557 32.658)>,
 <POINT (-116.843 32.6)>,
 <POINT (-116.682 32.837)>,
 <POINT (-116.924 33.224)>,
 <POINT (-116.621 33.226)>,
 <POINT (-116.748 32.865)>,
 <POINT (-116.71 33.069)>,
 <POINT (-116.468 32.594)>,
 <POINT (-116.69 33.087)>,
 <POINT (-117.383 33.429)>,
 <POINT (-116.424 32.634)>,
 <POINT (-116.779 32.798)>,
 <POINT (-116.63 32.85)>,
 <POINT (-116.634 32.865)>,
 <POINT (-116.992 32.997)>,
 <POINT (-116.873 33.276)>,
 <POINT (-116.697 33.248)>,
 <POINT (-116.631 32.648)>,
 <POINT (-116.913 33.301)>,
 <POINT (-116.828 33.031)>]

In [10]:
nam_within_vri_prediction_gpd = nam_within_vri_prediction_gpd[nam_within_vri_prediction_gpd['geometry'].isin(nam_points_to_update)]
nam_within_vri_prediction_gpd

,objectid,weatherstationcode,weatherstationname,scadartuid,structureid_left,nwszone,district_left,thomasbrospagegrid,constructionstatus,creationuser,...,snapshot_date,nam_geometry,station_geometry,polygon_geometry,nam_distance_from_station_km,month,day_of_year,abs_wind_speed_error,abs_wind_speed_error_pred,wind_speed_error_diff
770,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,...,2024-03-20,POINT (-116.920074 33.28915),POINT (-116.91293171066735 33.301346431011844),"MULTIPOLYGON Z (((-116.937336 33.301243 0, -11...",1.507430,10,288,23.673816,24.145582,0.471766
771,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,...,2024-03-20,POINT (-116.93619 33.302895),POINT (-116.91293171066735 33.301346431011844),"MULTIPOLYGON Z (((-116.937336 33.301243 0, -11...",2.172922,10,288,23.912602,23.089834,0.822768
772,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,...,2024-03-20,POINT (-116.91992 33.302765),POINT (-116.91293171066735 33.301346431011844),"MULTIPOLYGON Z (((-116.937336 33.301243 0, -11...",0.669590,10,288,24.257948,25.371410,1.113462
773,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,...,2024-03-20,POINT (-116.87143 33.275116),POINT (-116.91293171066735 33.301346431011844),"MULTIPOLYGON Z (((-116.819199 33.266861 0, -11...",4.838177,10,288,21.256195,22.722321,1.466126
774,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,...,2024-03-20,POINT (-116.85513 33.274963),POINT (-116.91293171066735 33.301346431011844),"MULTIPOLYGON Z (((-116.819199 33.266861 0, -11...",6.127915,10,288,20.120482,21.602160,1.481678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99788,146,HAU,Hauser Mountain,4828.0,Z972858,Mountain-258,4.0,1316-J1,A,calonzo,...,2024-03-20,POINT (-116.08725 32.666706),POINT (-116.5573052644581 32.658201312502996),"MULTIPOLYGON Z (((-116.392728 32.789193 0, -11...",44.104731,12,359,24.153124,28.415582,4.262458
99789,146,HAU,Hauser Mountain,4828.0,Z972858,Mountain-258,4.0,1316-J1,A,calonzo,...,2024-03-20,POINT (-116.07048 32.693657),POINT (-116.5573052644581 32.658201312502996),"MULTIPOLYGON Z (((-116.392728 32.789193 0, -11...",45.829921,12,359,24.135720,25.723161,1.587440
99790,146,HAU,Hauser Mountain,4828.0,Z972858,Mountain-258,4.0,1316-J1,A,calonzo,...,2024-03-20,POINT (-116.7005 32.729134),POINT (-116.5573052644581 32.658201312502996),"MULTIPOLYGON Z (((-116.392728 32.789193 0, -11...",15.562533,12,359,19.922061,28.153166,8.231105
99791,146,HAU,Hauser Mountain,4828.0,Z972858,Mountain-258,4.0,1316-J1,A,calonzo,...,2024-03-20,POINT (-116.05371 32.720608),POINT (-116.5573052644581 32.658201312502996),"MULTIPOLYGON Z (((-116.392728 32.789193 0, -11...",47.731163,12,359,24.191164,26.445676,2.254512


In [11]:
# Import necessary libraries
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from geopy.distance import geodesic
from shapely.ops import nearest_points
from sklearn.preprocessing import MinMaxScaler

# Load datasets (Assuming the data is already loaded as GeoDataFrames)
# Replace 'your_file_path' with the actual paths if needed
# gis_weather_station = gpd.read_file('your_file_path')
# windspeed_snapshot = gpd.read_file('your_file_path')
# src_vri_snapshot = gpd.read_file('your_file_path')
# nam_within_vri_prediction_gpd = gpd.read_file('your_file_path')

# nam_points_to_update = [
#     "POINT (-116.621796 32.592236)",
#     "POINT (-116.81482 32.64864)",
#     "POINT (-116.8631 32.662697)",
#     "POINT (-116.49263 32.590714)",
#     "POINT (-116.84729 32.63534)",
#     "POINT (-116.08725 32.666706)",
#     "POINT (-116.07048 32.693657)",
#     "POINT (-116.7005 32.729134)",
#     "POINT (-116.05371 32.720608)",
#     "POINT (-116.69971 32.783554)"
# ]
# # List of NAM points to update
# nam_points_to_update = [
#     "POINT (-116.08725 32.66671)",
#     "POINT (-116.05371 32.72061)",
#     "POINT (-116.07048 32.69366)",
#     "POINT (-116.69971 32.78355)",
#     "POINT (-116.66733 32.7832)",
#     "POINT (-116.81482 32.64864)",
#     "POINT (-116.8631 32.6627)",
#     "POINT (-116.88751 33.28887)",
#     "POINT (-116.45044 33.14809)",
#     "POINT (-116.84729 32.63534)",
#     "POINT (-116.68271 32.8378)",
#     "POINT (-116.92093 33.22111)",
#     "POINT (-116.4182 33.13408)",
#     "POINT (-116.36945 33.13344)",
#     "POINT (-116.33694 33.13301)",
#     "POINT (-116.32071 33.13279)",
#     "POINT (-116.7279 33.0696)",
#     "POINT (-116.7001 32.75634)",
#     "POINT (-116.30446 33.13256)",
#     "POINT (-116.93619 33.3029)"
# ]

# Convert NAM points to Shapely geometries
#nam_points_geom = nam_points_to_update

# Function to find the nearest alternative VRI polygon (excluding current)
def find_nearest_vri(nam_point, vri_polygons, current_vri_name):
    nam_coords = (nam_point.y, nam_point.x)  # (lat, lon)
    nearest_vri = None
    min_distance = float("inf")

    for _, row in vri_polygons.iterrows():
        if row['name'] == current_vri_name:
            continue  # Skip the current VRI polygon

        vri_centroid = row['geometry'].centroid
        vri_coords = (vri_centroid.y, vri_centroid.x)

        distance = geodesic(nam_coords, vri_coords).km  # Haversine distance in km
        if distance < min_distance:
            min_distance = distance
            nearest_vri = row

    return nearest_vri

In [12]:
# Create new columns with default value "no change"
nam_within_vri_prediction_gpd["New_VRI_Anemometer"] = "no change"
nam_within_vri_prediction_gpd["New_Polygon_Shape"] = "no change"
nam_within_vri_prediction_gpd

/home/khidayat/.local/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/khidayat/.local/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,objectid,weatherstationcode,weatherstationname,scadartuid,structureid_left,nwszone,district_left,thomasbrospagegrid,constructionstatus,creationuser,...,station_geometry,polygon_geometry,nam_distance_from_station_km,month,day_of_year,abs_wind_speed_error,abs_wind_speed_error_pred,wind_speed_error_diff,New_VRI_Anemometer,New_Polygon_Shape
770,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,...,POINT (-116.91293171066735 33.301346431011844),"MULTIPOLYGON Z (((-116.937336 33.301243 0, -11...",1.507430,10,288,23.673816,24.145582,0.471766,no change,no change
771,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,...,POINT (-116.91293171066735 33.301346431011844),"MULTIPOLYGON Z (((-116.937336 33.301243 0, -11...",2.172922,10,288,23.912602,23.089834,0.822768,no change,no change
772,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,...,POINT (-116.91293171066735 33.301346431011844),"MULTIPOLYGON Z (((-116.937336 33.301243 0, -11...",0.669590,10,288,24.257948,25.371410,1.113462,no change,no change
773,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,...,POINT (-116.91293171066735 33.301346431011844),"MULTIPOLYGON Z (((-116.819199 33.266861 0, -11...",4.838177,10,288,21.256195,22.722321,1.466126,no change,no change
774,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,...,POINT (-116.91293171066735 33.301346431011844),"MULTIPOLYGON Z (((-116.819199 33.266861 0, -11...",6.127915,10,288,20.120482,21.602160,1.481678,no change,no change
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99788,146,HAU,Hauser Mountain,4828.0,Z972858,Mountain-258,4.0,1316-J1,A,calonzo,...,POINT (-116.5573052644581 32.658201312502996),"MULTIPOLYGON Z (((-116.392728 32.789193 0, -11...",44.104731,12,359,24.153124,28.415582,4.262458,no change,no change
99789,146,HAU,Hauser Mountain,4828.0,Z972858,Mountain-258,4.0,1316-J1,A,calonzo,...,POINT (-116.5573052644581 32.658201312502996),"MULTIPOLYGON Z (((-116.392728 32.789193 0, -11...",45.829921,12,359,24.135720,25.723161,1.587440,no change,no change
99790,146,HAU,Hauser Mountain,4828.0,Z972858,Mountain-258,4.0,1316-J1,A,calonzo,...,POINT (-116.5573052644581 32.658201312502996),"MULTIPOLYGON Z (((-116.392728 32.789193 0, -11...",15.562533,12,359,19.922061,28.153166,8.231105,no change,no change
99791,146,HAU,Hauser Mountain,4828.0,Z972858,Mountain-258,4.0,1316-J1,A,calonzo,...,POINT (-116.5573052644581 32.658201312502996),"MULTIPOLYGON Z (((-116.392728 32.789193 0, -11...",47.731163,12,359,24.191164,26.445676,2.254512,no change,no change


In [13]:
# Apply nearest VRI assignment for the specific NAM points
for point_str in nam_points_to_update:
    current_vri_info = nam_within_vri_prediction_gpd.loc[
        nam_within_vri_prediction_gpd['geometry'] == point_str,
        ['polygon_geometry', 'name']
    ]

    if current_vri_info.empty:
        continue  # Skip if no current VRI info found

    current_vri_name = current_vri_info.iloc[0]['name']

    # Find the nearest alternative VRI polygon (excluding current)
    nearest_vri = find_nearest_vri(point_str, src_vri_snapshot, current_vri_name)

    if nearest_vri is not None:
        nam_within_vri_prediction_gpd.loc[
            nam_within_vri_prediction_gpd['geometry'] == point_str,
            ["New_VRI_Anemometer", "New_Polygon_Shape"]
        ] = [nearest_vri['anemometer'], nearest_vri['shape']]
nam_within_vri_prediction_gpd.head()

,objectid,weatherstationcode,weatherstationname,scadartuid,structureid_left,nwszone,district_left,thomasbrospagegrid,constructionstatus,creationuser,...,station_geometry,polygon_geometry,nam_distance_from_station_km,month,day_of_year,abs_wind_speed_error,abs_wind_speed_error_pred,wind_speed_error_diff,New_VRI_Anemometer,New_Polygon_Shape
770,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,...,POINT (-116.91293171066735 33.301346431011844),"MULTIPOLYGON Z (((-116.937336 33.301243 0, -11...",1.507430,10,288,23.673816,24.145582,0.471766,Rincon_VRI,"MULTIPOLYGON Z (((-116.944877 33.317623 0, -11..."
771,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,...,POINT (-116.91293171066735 33.301346431011844),"MULTIPOLYGON Z (((-116.937336 33.301243 0, -11...",2.172922,10,288,23.912602,23.089834,0.822768,Rincon_VRI,"MULTIPOLYGON Z (((-116.944877 33.317623 0, -11..."
772,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,...,POINT (-116.91293171066735 33.301346431011844),"MULTIPOLYGON Z (((-116.937336 33.301243 0, -11...",0.669590,10,288,24.257948,25.371410,1.113462,Rincon_VRI,"MULTIPOLYGON Z (((-116.944877 33.317623 0, -11..."
773,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,...,POINT (-116.91293171066735 33.301346431011844),"MULTIPOLYGON Z (((-116.819199 33.266861 0, -11...",4.838177,10,288,21.256195,22.722321,1.466126,Rincon_VRI,"MULTIPOLYGON Z (((-116.944877 33.317623 0, -11..."
774,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,...,POINT (-116.91293171066735 33.301346431011844),"MULTIPOLYGON Z (((-116.819199 33.266861 0, -11...",6.127915,10,288,20.120482,21.602160,1.481678,Rincon_VRI,"MULTIPOLYGON Z (((-116.944877 33.317623 0, -11..."


In [14]:
merged_wind_data = gis_weather_station_with_elevation_gpd.merge(
    windspeed_snapshot,
    left_on=["weatherstationcode"],  
    right_on=["station"],
    how="inner"
)


pd.set_option('display.max_columns', None)
# Spatial join to merge merged_wind_data with src_vri_snapshot
merged_wind_data = gpd.sjoin(merged_wind_data, src_vri_snapshot_gpd, predicate="within")
merged_wind_data

,objectid,weatherstationcode,weatherstationname,scadartuid,structureid_left,nwszone,district_left,thomasbrospagegrid,constructionstatus,creationuser,datecreated,datemodified,lastuser,structureguid,symbolrotation,latitude,longitude,elevation,twinguid,hftd_left,zone1idc_left,hftdidc_left,gdb_geomattr_data,globalid_left,shape_left,shape_srid_left,snapshot_date_x,geometry,station_elevation_m,date,wind_speed,station,snapshot_date_y,index_right,name,tessellate,extrude,visibility,globalid_right,anemometer,anemometercode,circuit,district_right,secdevice,structureid_right,tlid,gust_99pct,gust_95pct,gust_max,vri_risk,loaddate,county,hftd_right,hftdidc_right,zone1idc_right,globalid_1,shape_length,shape_area,shape_right,shape_srid_right,snapshot_date
0,1,CBD,Carlsbad,5158.0,P124785,Coastal-243,6.0,1126-G1,A,seu_gis_elec,00:00.0,00:00.0,bsingh,{64494E39-29E5-4911-ACB1-32A8CB6BCF2D},0,33.137352,-117.327141,72,NaN,1,N,N,NaN,{ED5DE7F2-F63A-41EA-BA80-9F71495334A3},POINT (1540299.9388 12029375.0425),4431,10/4/2024,POINT (-117.32717 33.13735),20.0,12/10/2012,11.0,CBD,8/2/2023,24,Carlsbad_VRI,-1,0,-1,{a5506318-6dfa-4de3-9df0-17deac78ed06},Carlsbad_VRI,CBD,NaN,NC,NaN,P124785,NaN,33.0,24.0,37.0,Medium,2021/05/21 10:37:46,SAN DIEGO,Non-HFTD,N,N,{C20D60A1-F6E8-4FD3-AF1F-C54B1A26E6DA},38417.323143,2.768495e+07,"MULTIPOLYGON Z (((-117.328519 33.134906 0, -11...",4326,2024-03-20
0,1,CBD,Carlsbad,5158.0,P124785,Coastal-243,6.0,1126-G1,A,seu_gis_elec,00:00.0,00:00.0,bsingh,{64494E39-29E5-4911-ACB1-32A8CB6BCF2D},0,33.137352,-117.327141,72,NaN,1,N,N,NaN,{ED5DE7F2-F63A-41EA-BA80-9F71495334A3},POINT (1540299.9388 12029375.0425),4431,10/4/2024,POINT (-117.32717 33.13735),20.0,12/10/2012,11.0,CBD,8/2/2023,287,TL23011_VRI,-1,0,-1,NaN,TL23011_VRI,NaN,NaN,NC;NE,NaN,NaN,23011.0,NaN,NaN,NaN,Medium,NaN,NaN,Non-HFTD,N,N,{06AB2161-9C7E-46A2-B25F-CDEC5983D2F2},80438.177003,9.097170e+06,"MULTIPOLYGON Z (((-117.334398 33.212038 0, -11...",4326,2024-03-20
1,1,CBD,Carlsbad,5158.0,P124785,Coastal-243,6.0,1126-G1,A,seu_gis_elec,00:00.0,00:00.0,bsingh,{64494E39-29E5-4911-ACB1-32A8CB6BCF2D},0,33.137352,-117.327141,72,NaN,1,N,N,NaN,{ED5DE7F2-F63A-41EA-BA80-9F71495334A3},POINT (1540299.9388 12029375.0425),4431,10/4/2024,POINT (-117.32717 33.13735),20.0,12/19/2012,16.0,CBD,8/2/2023,24,Carlsbad_VRI,-1,0,-1,{a5506318-6dfa-4de3-9df0-17deac78ed06},Carlsbad_VRI,CBD,NaN,NC,NaN,P124785,NaN,33.0,24.0,37.0,Medium,2021/05/21 10:37:46,SAN DIEGO,Non-HFTD,N,N,{C20D60A1-F6E8-4FD3-AF1F-C54B1A26E6DA},38417.323143,2.768495e+07,"MULTIPOLYGON Z (((-117.328519 33.134906 0, -11...",4326,2024-03-20
1,1,CBD,Carlsbad,5158.0,P124785,Coastal-243,6.0,1126-G1,A,seu_gis_elec,00:00.0,00:00.0,bsingh,{64494E39-29E5-4911-ACB1-32A8CB6BCF2D},0,33.137352,-117.327141,72,NaN,1,N,N,NaN,{ED5DE7F2-F63A-41EA-BA80-9F71495334A3},POINT (1540299.9388 12029375.0425),4431,10/4/2024,POINT (-117.32717 33.13735),20.0,12/19/2012,16.0,CBD,8/2/2023,287,TL23011_VRI,-1,0,-1,NaN,TL23011_VRI,NaN,NaN,NC;NE,NaN,NaN,23011.0,NaN,NaN,NaN,Medium,NaN,NaN,Non-HFTD,N,N,{06AB2161-9C7E-46A2-B25F-CDEC5983D2F2},80438.177003,9.097170e+06,"MULTIPOLYGON Z (((-117.334398 33.212038 0, -11...",4326,2024-03-20
2,1,CBD,Carlsbad,5158.0,P124785,Coastal-243,6.0,1126-G1,A,seu_gis_elec,00:00.0,00:00.0,bsingh,{64494E39-29E5-4911-ACB1-32A8CB6BCF2D},0,33.137352,-117.327141,72,NaN,1,N,N,NaN,{ED5DE7F2-F63A-41EA-BA80-9F71495334A3},POINT (1540299.9388 12029375.0425),4431,10/4/2024,POINT (-117.32717 33.13735),20.0,12/20/2012,17.0,CBD,8/2/2023,24,Carlsbad_VRI,-1,0,-1,{a5506318-6dfa-4de3-9df0-17deac78ed06},Carlsbad_VRI,CBD,NaN,NC,NaN,P124785,NaN,33.0,24.0,37.0,Medium,2021/05/21 10:37:46,SAN DIEGO,Non-HFTD,N,N,{C20D60A1-F6E8-4FD3-AF1F-C54B1A26E6DA},38417.323143,2.768495e+07,"MULTIPOLYGON Z (((-117.328519 33.134906 0, -11...",4326,2024-03-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [15]:
nam_within_vri_prediction_gpd['New_VRI_Anemometer'].unique()

array(['Rincon_VRI', 'Dye Mountain_VRI', 'East Warners_VRI',
       'Cuca Ranch_VRI', 'Iron Mountain Trail_VRI',
       'Santa Ysabel North_VRI', 'Loveland_VRI',
       'Viejas Mountain Trail_VRI', 'Warners_VRI', 'Descanso Sub _VRI',
       'TL6923_VRI', 'West Descanso_VRI', 'Pacific Crest Trail_VRI',
       'Viejas_VRI', 'Paradise Mountain_VRI', 'De Luz Creek_VRI',
       'Country Estates_VRI', 'TL23040_VRI', 'Morena Dam_VRI'],
      dtype=object)

In [16]:
#merged_wind_data_gdf = convert_to_gdf(merged_wind_data)
df = nam_within_vri_prediction_gpd[['geometry', 'abs_wind_speed_error']]
df_filtered = df[df['geometry'].isin(nam_points_to_update)]
df_filtered.groupby('geometry').mean()

,abs_wind_speed_error
geometry,
POINT (-116.99221 32.99657),17.573553
POINT (-117.38281 33.42929),19.733541
POINT (-116.91293 33.30135),17.041791
POINT (-116.92351 33.22445),23.202737
POINT (-116.82777 33.03051),16.767753
POINT (-116.70992 33.06884),20.362472
POINT (-116.68973 33.08678),20.004840
POINT (-116.87335 33.27619),17.471252
POINT (-116.69718 33.24791),17.356894


In [17]:
filtered_nam_within_vri_subset = nam_within_vri_prediction_gpd[
    nam_within_vri_prediction_gpd['geometry'].isin(nam_points_to_update)
]
filtered_nam_within_vri_subset

,objectid,weatherstationcode,weatherstationname,scadartuid,structureid_left,nwszone,district_left,thomasbrospagegrid,constructionstatus,creationuser,datecreated,datemodified,lastuser,structureguid,symbolrotation,latitude_left,longitude_left,elevation,twinguid,hftd_left,zone1idc_left,hftdidc_left,gdb_geomattr_data,globalid_left,shape_left,shape_srid_left,snapshot_date_x,geometry,station_elevation_m,station_date,station_wind_speed,snapshot_date_y,latitude_right,longitude_right,nam_date,nam_wind_speed,nam_elevation_m,name,tessellate,extrude,visibility,globalid_right,anemometer,anemometercode,circuit,district_right,secdevice,structureid_right,tlid,gust_99pct,gust_95pct,gust_max,vri_risk,loaddate,county,hftd_right,hftdidc_right,zone1idc_right,globalid_1,shape_length,shape_area,polygon_shape,shape_srid_right,snapshot_date,nam_geometry,station_geometry,polygon_geometry,nam_distance_from_station_km,month,day_of_year,abs_wind_speed_error,abs_wind_speed_error_pred,wind_speed_error_diff,New_VRI_Anemometer,New_Polygon_Shape
770,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,00:00.0,00:00.0,bsingh,{04BAE46A-10DE-41EE-8AFD-B46BE90E5E6F},0,33.301305,-116.912993,2513,NaN,3,N,Y,NaN,{82759049-437B-4A5C-8796-10C8D7098C5B},POINT (1667010.27856 12088881.1767),4431,10/4/2024,POINT (-116.91293 33.30135),749.0,10/14/2012,27.0,8/2/2023,33.289150,-116.920074,10/14/2012,3.326184,715.0,Cuca Ranch_VRI,-1,0,-1,{d8e3b1dd-aa84-4413-a4f5-91f45f2bd841},Cuca Ranch_VRI,CCR,NaN,NE,NaN,Z118027,NaN,47.0,34.0,61.0,Medium,2021/05/21 10:37:46,SAN DIEGO,Tier 3,Y,N,{425B9034-E2AF-4A09-927E-3C2FC602F282},19938.846484,9.575396e+06,"MULTIPOLYGON Z (((-116.937336 33.301243 0, -11...",4326,2024-03-20,POINT (-116.920074 33.28915),POINT (-116.91293171066735 33.301346431011844),"MULTIPOLYGON Z (((-116.937336 33.301243 0, -11...",1.507430,10,288,23.673816,24.145582,0.471766,Rincon_VRI,"MULTIPOLYGON Z (((-116.944877 33.317623 0, -11..."
771,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,00:00.0,00:00.0,bsingh,{04BAE46A-10DE-41EE-8AFD-B46BE90E5E6F},0,33.301305,-116.912993,2513,NaN,3,N,Y,NaN,{82759049-437B-4A5C-8796-10C8D7098C5B},POINT (1667010.27856 12088881.1767),4431,10/4/2024,POINT (-116.91293 33.30135),749.0,10/14/2012,27.0,8/2/2023,33.302895,-116.936190,10/14/2012,3.087398,541.0,Cuca Ranch_VRI,-1,0,-1,{d8e3b1dd-aa84-4413-a4f5-91f45f2bd841},Cuca Ranch_VRI,CCR,NaN,NE,NaN,Z118027,NaN,47.0,34.0,61.0,Medium,2021/05/21 10:37:46,SAN DIEGO,Tier 3,Y,N,{425B9034-E2AF-4A09-927E-3C2FC602F282},19938.846484,9.575396e+06,"MULTIPOLYGON Z (((-116.937336 33.301243 0, -11...",4326,2024-03-20,POINT (-116.93619 33.302895),POINT (-116.91293171066735 33.301346431011844),"MULTIPOLYGON Z (((-116.937336 33.301243 0, -11...",2.172922,10,288,23.912602,23.089834,0.822768,Rincon_VRI,"MULTIPOLYGON Z (((-116.944877 33.317623 0, -11..."
772,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,00:00.0,00:00.0,bsingh,{04BAE46A-10DE-41EE-8AFD-B46BE90E5E6F},0,33.301305,-116.912993,2513,NaN,3,N,Y,NaN,{82759049-437B-4A5C-8796-10C8D7098C5B},POINT (1667010.27856 12088881.1767),4431,10/4/2024,POINT (-116.91293 33.30135),749.0,10/14/2012,27.0,8/2/2023,33.302765,-116.919920,10/14/2012,2.742052,662.0,Cuca Ranch_VRI,-1,0,-1,{d8e3b1dd-aa84-4413-a4f5-91f45f2bd841},Cuca Ranch_VRI,CCR,NaN,NE,NaN,Z118027,NaN,47.0,34.0,61.0,Medium,2021/05/21 10:37:46,SAN DIEGO,Tier 3,Y,N,{425B9034-E2AF-4A09-927E-3C2FC602F282},19938.846484,9.575396e+06,"MULTIPOLYGON Z (((-116.937336 33.301243 0, -11...",4326,2024-03-20,POINT (-116.91992 33.302765),POINT (-116.91293171066735 33.301346431011844),"MULTIPOLYGON Z (((-116.937336 33.301243 0, -11...",0.669590,10,288,24.257948,25.371410,1.113462,Rincon_VRI,"MULTIPOLYGON Z (((-116.944877 33.317623 0, -11..."
773,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,00:00.0,00:00.0,bsingh,{04BAE46A-10DE-41EE-8AFD-B46BE90E5E6F},0,33.301305,-116.912993,2513,NaN,3,N,Y,NaN,{82759049-437B-4A5C-8796-10C8D7098C5B},POINT (166

In [18]:
merged_filtered_wind_data = filtered_nam_within_vri_subset.merge(
    merged_wind_data,
    left_on=["New_VRI_Anemometer", "nam_date"],  # Columns from filtered_nam_within_vri_subset
    right_on=["anemometer", "date"],            # Columns from merged_wind_data
    how="left"  # Keep all rows from filtered_nam_within_vri_subset
)

In [19]:
merged_filtered_wind_data.head()

,objectid_x,weatherstationcode_x,weatherstationname_x,scadartuid_x,structureid_left_x,nwszone_x,district_left_x,thomasbrospagegrid_x,constructionstatus_x,creationuser_x,datecreated_x,datemodified_x,lastuser_x,structureguid_x,symbolrotation_x,latitude_left,longitude_left,elevation_x,twinguid_x,hftd_left_x,zone1idc_left_x,hftdidc_left_x,gdb_geomattr_data_x,globalid_left_x,shape_left_x,shape_srid_left_x,snapshot_date_x_x,geometry_x,station_elevation_m_x,station_date,station_wind_speed,snapshot_date_y_x,latitude_right,longitude_right,nam_date,nam_wind_speed,nam_elevation_m,name_x,tessellate_x,extrude_x,visibility_x,globalid_right_x,anemometer_x,anemometercode_x,circuit_x,district_right_x,secdevice_x,structureid_right_x,tlid_x,gust_99pct_x,gust_95pct_x,gust_max_x,vri_risk_x,loaddate_x,county_x,hftd_right_x,hftdidc_right_x,zone1idc_right_x,globalid_1_x,shape_length_x,shape_area_x,polygon_shape,shape_srid_right_x,snapshot_date_x,nam_geometry,station_geometry,polygon_geometry,nam_distance_from_station_km,month,day_of_year,abs_wind_speed_error,abs_wind_speed_error_pred,wind_speed_error_diff,New_VRI_Anemometer,New_Polygon_Shape,objectid_y,weatherstationcode_y,weatherstationname_y,scadartuid_y,structureid_left_y,nwszone_y,district_left_y,thomasbrospagegrid_y,constructionstatus_y,creationuser_y,datecreated_y,datemodified_y,lastuser_y,structureguid_y,symbolrotation_y,latitude,longitude,elevation_y,twinguid_y,hftd_left_y,zone1idc_left_y,hftdidc_left_y,gdb_geomattr_data_y,globalid_left_y,shape_left_y,shape_srid_left_y,snapshot_date_x_y,geometry_y,station_elevation_m_y,date,wind_speed,station,snapshot_date_y_y,index_right,name_y,tessellate_y,extrude_y,visibility_y,globalid_right_y,anemometer_y,anemometercode_y,circuit_y,district_right_y,secdevice_y,structureid_right_y,tlid_y,gust_99pct_y,gust_95pct_y,gust_max_y,vri_risk_y,loaddate_y,county_y,hftd_right_y,hftdidc_right_y,zone1idc_right_y,globalid_1_y,shape_length_y,shape_area_y,shape_right,shape_srid_right_y,snapshot_date_y
0,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,00:00.0,00:00.0,bsingh,{04BAE46A-10DE-41EE-8AFD-B46BE90E5E6F},0,33.301305,-116.912993,2513,NaN,3,N,Y,NaN,{82759049-437B-4A5C-8796-10C8D7098C5B},POINT (1667010.27856 12088881.1767),4431,10/4/2024,POINT (-116.91293 33.30135),749.0,10/14/2012,27.0,8/2/2023,33.289150,-116.920074,10/14/2012,3.326184,715.0,Cuca Ranch_VRI,-1,0,-1,{d8e3b1dd-aa84-4413-a4f5-91f45f2bd841},Cuca Ranch_VRI,CCR,NaN,NE,NaN,Z118027,NaN,47.0,34.0,61.0,Medium,2021/05/21 10:37:46,SAN DIEGO,Tier 3,Y,N,{425B9034-E2AF-4A09-927E-3C2FC602F282},19938.846484,9.575396e+06,"MULTIPOLYGON Z (((-116.937336 33.301243 0, -11...",4326,2024-03-20,POINT (-116.920074 33.28915),POINT (-116.91293171066735 33.301346431011844),"MULTIPOLYGON Z (((-116.937336 33.301243 0, -11...",1.507430,10,288,23.673816,24.145582,0.471766,Rincon_VRI,"MULTIPOLYGON Z (((-116.944877 33.317623 0, -11...",101.0,RIN,Rincon,1985.0,P214207,Inland Valley-250,3.0,1051-E7,A,seu_gis_elec,00:00.0,24:08.0,eaguila1,{3AAA6B77-490E-4D29-86D6-B9C078008BFB},0.0,33.28779,-116.95642,1063.0,NaN,3.0,N,Y,NaN,{1EC2E1BF-5B21-4ACD-AFAC-70F2E3094B87},POINT (1653736.08624 12083945.0878),4431.0,10/4/2024,POINT (-116.9564 33.2878),321.0,10/14/2012,14.0,RIN,8/2/2023,150.0,Rincon_VRI,-1.0,0.0,-1.0,{51af3ee7-7a9b-49f1-8813-c40acc26a7ba},Rincon_VRI,RIN,NaN,NE,NaN,P214207,NaN,36.0,28.0,44.0,Medium,2021/05/21 10:37:46,SAN DIEGO,Tier 3,Y,N,{5C997B8F-97CB-440B-AD8D-ADB054CF42E8},13882.748672,9.625336e+06,"MULTIPOLYGON Z (((-116.944877 33.317623 0, -11...",4326.0,2024-03-20
1,2,CCR,Cuca Ranch,1915.0,Z118027,Mountain-258,3.0,1052-A5,A,seu_gis_elec,00:00.0,00:00.0,bsingh,{04BAE46A-10DE-41EE-8AFD-B46BE90E5E6F},0,33.301305,-116.912993,2513,NaN,3,N,Y,NaN,{82759049-437B-4A5C-8796-10C8D7098C5B},POINT (1667010.27856 12088881.1767),4431,10/4/2024,POINT (-116.91293 33.30135),749.0,10/14/2012,27.0,8/2/2023,33.302895,-116.936190,10/14/2012,3.087398,541.0,Cuca Ranch_VRI,-1,0,-1,{d8e3b1dd-aa84-4413-a4f5-91f45f2bd841},Cuca Ranch_

In [20]:
# Select relevant columns for calculation
important_columns = [
    "geometry_x",
    "nam_wind_speed",
    "New_VRI_Anemometer",
    "wind_speed"
]

filtered_merged_wind_data = merged_filtered_wind_data[important_columns]

# Filter to only include rows where New_VRI_Anemometer has changed
filtered_merged_wind_data = filtered_merged_wind_data[
    filtered_merged_wind_data["New_VRI_Anemometer"] != "no change"
]

# Calculate new absolute wind speed error
filtered_merged_wind_data["new_abs_wind_speed_error"] = (
    filtered_merged_wind_data["nam_wind_speed"] - filtered_merged_wind_data["wind_speed"]
).abs()

filtered_merged_wind_data

,geometry_x,nam_wind_speed,New_VRI_Anemometer,wind_speed,new_abs_wind_speed_error
0,POINT (-116.91293 33.30135),3.326184,Rincon_VRI,14.0,10.673816
1,POINT (-116.91293 33.30135),3.087398,Rincon_VRI,14.0,10.912602
2,POINT (-116.91293 33.30135),2.742052,Rincon_VRI,14.0,11.257948
3,POINT (-116.91293 33.30135),5.743805,Rincon_VRI,14.0,8.256195
4,POINT (-116.91293 33.30135),6.879518,Rincon_VRI,14.0,7.120482
...,...,...,...,...,...
5422,POINT (-116.55731 32.6582),4.846876,Morena Dam_VRI,30.0,25.153124
5423,POINT (-116.55731 32.6582),4.864280,Morena Dam_VRI,30.0,25.135720
5424,POINT (-116.55731 32.6582),9.077939,Morena Dam_VRI,30.0,20.922061
5425,POINT (-116.55731 32.6582),4.808835,Morena Dam_VRI,30.0,25.191164


In [21]:
df_filtered

,geometry,abs_wind_speed_error
770,POINT (-116.91293 33.30135),23.673816
771,POINT (-116.91293 33.30135),23.912602
772,POINT (-116.91293 33.30135),24.257948
773,POINT (-116.91293 33.30135),21.256195
774,POINT (-116.91293 33.30135),20.120482
...,...,...
99788,POINT (-116.55731 32.6582),24.153124
99789,POINT (-116.55731 32.6582),24.135720
99790,POINT (-116.55731 32.6582),19.922061
99791,POINT (-116.55731 32.6582),24.191164


In [22]:
# Group both DataFrames by 'nam_geometry' and compute mean errors
new_error_df = filtered_merged_wind_data.groupby("geometry_x")[["new_abs_wind_speed_error"]].mean().reset_index()
old_error_df = df_filtered.groupby("geometry")[["abs_wind_speed_error"]].mean().reset_index()

# Merge both DataFrames to compare old vs new error
comparison_df = old_error_df.merge(new_error_df, left_on='geometry', right_on='geometry_x').drop('geometry_x', axis=1)

# Calculate the difference (improvement)
comparison_df["error_difference"] = comparison_df["abs_wind_speed_error"] - comparison_df["new_abs_wind_speed_error"]

In [23]:
comparison_df

,geometry,abs_wind_speed_error,new_abs_wind_speed_error,error_difference
0,POINT (-116.99221 32.99657),17.573553,14.527041,3.046512
1,POINT (-117.38281 33.42929),19.733541,9.475635,10.257906
2,POINT (-116.91293 33.30135),17.041791,9.535630,7.506161
3,POINT (-116.92351 33.22445),23.202737,7.817333,15.385405
4,POINT (-116.82777 33.03051),16.767753,10.222838,6.544915
5,POINT (-116.70992 33.06884),20.362472,20.362472,0.000000
6,POINT (-116.68973 33.08678),20.004840,12.082884,7.921956
7,POINT (-116.87335 33.27619),17.471252,15.241698,2.229554
8,POINT (-116.69718 33.24791),17.356894,NaN,NaN
9,POINT (-116.62066 33.22641),22.624386,22.624386,0.000000


In [24]:
# Extract new VRI polygon information
filtered_merged_wind_data.rename(columns={'geometry_x': 'geometry'}, inplace=True)
new_vri_info = filtered_merged_wind_data[["geometry", "New_VRI_Anemometer"]].drop_duplicates()

# Merge to include the new VRI polygons
comparison_df = comparison_df.merge(new_vri_info, on="geometry", how="left")

# Filter for optimal geometries where the error difference is positive (improvement)
optimum_geometry = comparison_df[comparison_df["error_difference"] > 0]

# Rename column for clarity
optimum_geometry.rename(columns={"New_VRI_Anemometer": "New_VRI_Polygon"}, inplace=True)
optimum_geometry

/tmp/ipykernel_1683/3242496136.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  optimum_geometry.rename(columns={"New_VRI_Anemometer": "New_VRI_Polygon"}, inplace=True)


,geometry,abs_wind_speed_error,new_abs_wind_speed_error,error_difference,New_VRI_Polygon
0,POINT (-116.99221 32.99657),17.573553,14.527041,3.046512,Iron Mountain Trail_VRI
1,POINT (-117.38281 33.42929),19.733541,9.475635,10.257906,De Luz Creek_VRI
2,POINT (-116.91293 33.30135),17.041791,9.535630,7.506161,Rincon_VRI
3,POINT (-116.92351 33.22445),23.202737,7.817333,15.385405,Paradise Mountain_VRI
4,POINT (-116.82777 33.03051),16.767753,10.222838,6.544915,Country Estates_VRI
6,POINT (-116.68973 33.08678),20.004840,12.082884,7.921956,Santa Ysabel North_VRI
7,POINT (-116.87335 33.27619),17.471252,15.241698,2.229554,Cuca Ranch_VRI
12,POINT (-116.74794 32.86471),20.960205,17.756502,3.203703,Viejas Mountain Trail_VRI
13,POINT (-116.6825 32.83679),23.736281,12.542942,11.193339,Viejas_VRI
14,POINT (-116.77918 32.79763),18.621774,15.598794,3.022980,Loveland_VRI
